In [2]:
import os
import cv2

def crop_crossarm_area(image, label_line, top_expand=0.2, crop_height_ratio=0.50):
    H, W = image.shape[:2]
    values = list(map(float, label_line.strip().split()))
    if len(values) < 5:
        raise ValueError("Invalid label format")
    class_id, x_c, y_c, w, h = values[:5]

    # Convert normalized YOLO coords to pixel values
    x_center = x_c * W
    y_center = y_c * H
    box_width = w * W
    box_height = h * H

    # Original top-left of the box
    x1 = x_center - box_width / 2
    y1 = y_center - box_height / 2

    # Compute vertical crop bounds
    crop_y1 = max(0, y1 - top_expand * box_height)
    crop_y2 = min(H, y1 + crop_height_ratio * box_height)

    # Compute horizontal crop bounds using box_height
    half_crop_width = 0.57 * box_height
    crop_x1 = max(0, x_center - half_crop_width)
    crop_x2 = min(W, x_center + half_crop_width)

    # Perform cropping
    return image[int(crop_y1):int(crop_y2), int(crop_x1):int(crop_x2)]



def process_directory(image_dir, label_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(image_dir):
        if not filename.lower().endswith((".jpg", ".png")):
            continue

        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")

        # Skip if label file doesn't exist
        if not os.path.exists(label_path):
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load {filename}, skipping.")
            continue

        # Read all valid YOLO label lines
        with open(label_path, 'r') as f:
            lines = [line for line in f if len(line.strip().split()) >= 5]

        if not lines:
            continue

        base_name = os.path.splitext(filename)[0]

        # Crop and save for each bounding box
        for idx, label_line in enumerate(lines):
            try:
                cropped = crop_crossarm_area(image, label_line)
                save_name = f"{base_name}_crop_{idx}.jpg"
                save_path = os.path.join(output_dir, save_name)
                cv2.imwrite(save_path, cropped)
            except Exception as e:
                print(f"Skipping {filename} bbox {idx} due to error: {e}")

if __name__ == "__main__":
    process_directory(
        image_dir="/home/line_quality/line_quality/Sampath/Models/dino_crop_rejection/Classification/Rejected/",       # Folder with .jpg/.png images
        label_dir="/home/line_quality/line_quality/Sampath/Models/dino_crop_rejection/OutputPreds_rejected/predict/labels/",       # Folder with corresponding YOLO .txt labels
        output_dir="/home/line_quality/line_quality/Sampath/Models/dino_crop_rejection/OutputCropsRejected/" # Folder to save all cropped areas
    )
    print("Cropping for all bounding boxes finished.")


Cropping for all bounding boxes finished.
